In [1]:
# Imports
import copy
import os
import tempfile

import numpy as np
import time
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from pprint import pprint

from c3.c3objs import Quantity as Qty
from c3.optimizers.optimalcontrol import OptimalControl
from c3.parametermap import ParameterMap
from c3.experiment import Experiment
from c3.model import Model
from c3.generator.generator import Generator

import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

import c3.libraries.algorithms as algorithms
import c3.libraries.hamiltonians as hamiltonians
import c3.libraries.fidelities as fidelities
import c3.libraries.envelopes as envelopes
import c3.utils.qt_utils as qt_utils
import c3.utils.tf_utils as tf_utils

2022-05-23 10:22:34.690466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 10:22:34.690494: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


The model consists of a single device with levels and a drive line.

In [2]:
# Model components
qubit_levels = 3
frequency = 5e9
anharmonicity = -210e6
qubit_temp = 0

qubit = chip.Qubit(
    name="Q",
    desc="Qubit",
    freq=Qty(value=frequency, min_val=4.995e9, max_val=5.005e9, unit="Hz 2pi"),
    anhar=Qty(value=anharmonicity, min_val=-380e6, max_val=-120e6, unit="Hz 2pi"),
    hilbert_dim=qubit_levels,
    temp=Qty(value=qubit_temp, min_val=0.0, max_val=0.12, unit="K"),
)

drive = chip.Drive(
    name="drive",
    desc="Drive",
    comment="Drive line on qubit",
    connected=["Q"],
    hamiltonian_func=hamiltonians.x_drive,
)

model = Model([qubit], [drive])
model.set_lindbladian(False)
model.set_dressed(True)

2022-05-23 10:22:37.080278: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 10:22:37.080312: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-23 10:22:37.080335: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist


This creates the typical generator configuration

In [3]:
sim_res = 100e9
awg_res = 2e9
generator = Generator(
    devices={
        "LO": devices.LO(name="lo", resolution=sim_res, outputs=1),
        "AWG": devices.AWG(name="awg", resolution=awg_res, outputs=1),
        "DigitalToAnalog": devices.DigitalToAnalog(
            name="dac", resolution=sim_res, inputs=1, outputs=1
        ),
        "Response": devices.ResponseFFT(
            name="resp",
            rise_time=Qty(value=0.3e-9, min_val=0.05e-9, max_val=0.6e-9, unit="s"),
            resolution=sim_res,
            inputs=1,
            outputs=1,
        ),
        "Mixer": devices.Mixer(name="mixer", inputs=2, outputs=1),
        "VoltsToHertz": devices.VoltsToHertz(
            name="v_to_hz",
            V_to_Hz=Qty(value=1e9, min_val=0.9e9, max_val=1.1e9, unit="Hz/V"),
            inputs=1,
            outputs=1,
        ),
    },
    chains={
        "drive": {
            "LO": [],
            "AWG": [],
            "DigitalToAnalog": ["AWG"],
            "Response": ["DigitalToAnalog"],
            "Mixer": ["LO", "Response"],
            "VoltsToHertz": ["Mixer"],
        }
    },
)

The signal carrier is resonant to the qubit frequency. The envelope is piece-wise constant
initialised with a Gaussian shape.

In [4]:
# Create a piece-wise gaussian shape
numPWCPieces = 20
t_final = 10e-9
sigma = t_final / 5
times = tf.linspace(0.0, t_final, numPWCPieces)
gaussian_shape = tf.exp(-((times - t_final / 2) ** 2) / (2 * sigma ** 2))

# Define the envelope
gauss_params = {
    "amp": Qty(value=0.45, min_val=0.4, max_val=0.6, unit="V"),
    "t_final": Qty(
        value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s"
    ),
    "sigma": Qty(
        value=t_final / 4, min_val=t_final / 8, max_val=t_final / 2, unit="s"
    ),
    "xy_angle": Qty(
        value=0.0, min_val=-0.5 * np.pi, max_val=2.5 * np.pi, unit="rad"
    ),
    "freq_offset": Qty(
        value=-50e6,
        min_val=-60e6,
        max_val=-40e6,
        unit="Hz 2pi",
    ),
    "delta": Qty(value=-1, min_val=-5, max_val=3, unit=""),
    "t_bin_start": Qty(0.0),
    "t_bin_end": Qty(t_final),
    "inphase": Qty(gaussian_shape),
    "quadrature": Qty(tf.zeros_like(gaussian_shape))
}

gauss_envelope = pulse.EnvelopeDrag(
    name="gauss",
    desc="Gaussian comp for single-qubit gates",
    params=gauss_params,
    shape=envelopes.pwc,
)

# Define the carrier
carrier_parameters = {
    "freq": Qty(value=frequency, min_val=4.5e9, max_val=6e9, unit="Hz 2pi"),
    "framechange": Qty(value=0.0, min_val=-np.pi, max_val=3 * np.pi, unit="rad"),
}
carrier = pulse.Carrier(
    name="carrier",
    desc="Frequency of the local oscillator",
    params=carrier_parameters,
)

The goal is to optimise the PWC envelope to facilitate an $X(\pi/2)$ gate.

In [5]:
rx90p = gates.Instruction(
    name="rx90p", t_start=0.0, t_end=t_final, channels=["drive"], targets=[0]
)
rx90p.add_component(gauss_envelope, "drive")
rx90p.add_component(carrier, "drive")

This sets up the experiment and computes the initial propagator.

In [6]:
parameter_map = ParameterMap(
    instructions=[rx90p], model=model, generator=generator
)
exp = Experiment(pmap=parameter_map)
exp.set_opt_gates([rx90p.get_key()])
exp.compute_propagators()

{'rx90p[0]': <tf.Tensor: shape=(3, 3), dtype=complex128, numpy=
 array([[ 0.69137933+0.2553401j , -0.62511455+0.11198216j,
          0.18820308-0.13441523j],
        [-0.62440622+0.11395454j, -0.56593468+0.51656201j,
          0.02184086+0.0976516j ],
        [ 0.12619595-0.19494069j,  0.05574723+0.08040937j,
          0.5207599 +0.8156644j ]])>}

Define all parameters that the optimiser can change.

In [7]:
opt_map = [
    [(rx90p.get_key(), "drive", gauss_envelope.name, "amp")],
    [(rx90p.get_key(), "drive", gauss_envelope.name, "inphase")],
    [(rx90p.get_key(), "drive", gauss_envelope.name, "quadrature")],
    [(rx90p.get_key(), "drive", gauss_envelope.name, "xy_angle")],
    [(rx90p.get_key(), "drive", gauss_envelope.name, "t_final")],
    [(rx90p.get_key(), "drive", carrier.name, "freq")],
    [(rx90p.get_key(), "drive", carrier.name, "framechange")],
]
parameter_map.set_opt_map(opt_map)
parameter_map.print_parameters()

rx90p[0]-drive-gauss-amp              : 450.000 mV 
rx90p[0]-drive-gauss-inphase          : 0.0439  0.0819  0.143  0.232  0.351  0.496  0.654  0.805  0.925  0.991  0.991  0.925  0.805  0.654  0.496  0.351  0.232  0.143  0.0819  0.0439  
rx90p[0]-drive-gauss-quadrature       : 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
rx90p[0]-drive-gauss-xy_angle         : -444.089 arad 
rx90p[0]-drive-gauss-t_final          : 10.000 ns 
rx90p[0]-drive-carrier-freq           : 5.000 GHz 2pi 
rx90p[0]-drive-carrier-framechange    : 0.000 rad 



Set up the optimiser and run it.

In [8]:
log_dir = os.path.join(tempfile.TemporaryDirectory().name, "c3logs")
opt = OptimalControl(
    dir_path=log_dir,
    fid_func=fidelities.unitary_infid_set,
    fid_subspace=[qubit.name],
    pmap=exp.pmap,
    algorithm=algorithms.lbfgs,
    options={"maxfun": 1000, "ftol": 1e-6},
    run_name="X-gate optimisation"
)
opt.set_exp(exp)
opt.optimize_controls()

print(opt.current_best_goal)
exp.pmap.print_parameters()

C3:STATUS:Saving as: /tmp/tmp7n4nkr9n/c3logs/X-gate optimisation/2022_05_23_T_10_22_37/open_loop.c3log


/home/user/c3/c3/optimizers/optimizer.py:296: Warning: rx90p[0]-drive-gauss-quadrature has no gradient. This might indicate no usage for current experiment.
  warnings.warn(
/home/user/c3/c3/optimizers/optimizer.py:296: Warning: rx90p[0]-drive-gauss-t_final has no gradient. This might indicate no usage for current experiment.
  warnings.warn(


0.009864424170508923
rx90p[0]-drive-gauss-amp              : 599.998 mV 
rx90p[0]-drive-gauss-inphase          : 0.0425  0.0414  1.09  0.109  0.166  1.09  0.0396  0.675  1.09  0.0426  1.08  1.07  0.0395  1.09  0.774  0.0417  1.09  0.1  0.0409  1.01  
rx90p[0]-drive-gauss-quadrature       : 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
rx90p[0]-drive-gauss-xy_angle         : -73.594 mrad 
rx90p[0]-drive-gauss-t_final          : 10.000 ns 
rx90p[0]-drive-carrier-freq           : 5.761 GHz 2pi 
rx90p[0]-drive-carrier-framechange    : -672.546 mrad 

